In [1]:
import numpy as np
import pandas as pd
import statistics
import statsmodels.api as sm
import scipy.optimize as opt
import matplotlib.pyplot as plt

In [5]:
p3.9 -m pip install openpyxl

SyntaxError: invalid syntax (<ipython-input-5-d0c341c9c751>, line 1)

In [6]:
data=pd.read_excel('RatesComma.xlsx')

In [7]:
g_data = data['Group 12']
history = []
for i in g_data:
    history.append(i)

In [8]:
def calculate_mse(alpha, history):
    n = len(history)
    r = np.mean(history)
    sum_diff = np.sum((history[1:] - r) * (history[:-1] - r))
    sum_var = np.sum((history[:-1] - r) ** 2)
    mse = (sum_diff / sum_var) - alpha
    return mse
initial_alpha = 0.2
result = opt.root(calculate_mse, initial_alpha, history)
estimated_alpha = result.x[0]
print(estimated_alpha)      #Speed of mean reversion calculation

0.9974432841855414


In [9]:
def simulate_vasicek(T, N, history, estimated_alpha):
    r0 = history[-1]
    alpha = estimated_alpha
    mu = np.mean(np.array(history))     #Estimated mean of data
    sigma = np.std(np.array(history))    #Estimated volatility of data
    dt = T / N
    Z = np.random.standard_normal(N)
    r = np.zeros(N)
    r[0] = r0

    for i in range(1, N):
        drift = alpha * (mu - r[i-1]) * dt
        diffusion = sigma * np.sqrt(dt) * Z[i]
        r[i] = r[i-1] + drift + diffusion

    return r


In [10]:
def find_future_value(simulated_rates, notional, fixed_rate, floating_spread, time_in_days):
    present_values = []
    for i in range(time_in_days):
        present_values.append(0)
    for i in range(time_in_days):
        fixed_cashflows = fixed_rate * notional
        floating_cashflows = (simulated_rates[i] + floating_spread) * notional
        present_values[i] = fixed_cashflows - floating_cashflows
    future_value = np.mean(present_values)
    return future_value, present_values

In [11]:
def calculate_exposure_metrics(list_future_values):
#     n = len(future_values)
#     exposures = np.zeros(n)
#     peak_exposures = np.zeros(n)
#     max_peak_exposure = 0.0

#     for i in range(n):
#         if i == 0:
#             exposures[i] = future_values[i]
#             peak_exposures[i] = exposures[i]
#         else:
#             exposures[i] = exposures[i-1] + future_values[i]
#             peak_exposures[i] = max(exposures[i], peak_exposures[i-1])

#         max_peak_exposure = max(max_peak_exposure, peak_exposures[i])

#     expected_exposure = np.mean(exposures)
#     percentile_975 = np.percentile(peak_exposures, 97.5)

#     return expected_exposure, percentile_975, max_peak_exposure
    
    no_traj = len(list_future_values)
    traj_len = len(list_future_values[0])
    
    peak_exposure = np.zeros(traj_len)
    exp_exposure = np.zeros(traj_len)
    for i in range(traj_len):
        temp_store = []
        for j in range(no_traj):
            temp_store.append(max(list_future_values[j][i],0))
        exp_exposure[i] = 1.0*sum(temp_store)/no_traj
        peak_exposure[i] = float(np.percentile(np.array(temp_store),97.5))
    return exp_exposure,peak_exposure,max(peak_exposure)

In [226]:
#For 10 year interest swap with a notional value of 2 million, swap rate of 2%.
trajectories = 4
values_10 = []
future_values_10 = []
for i in range(trajectories):
    simulated_rates = simulate_vasicek(10, 2520, history, estimated_alpha)
    future_value, values = find_future_value(simulated_rates, 2000000, 0.02, 0.0003,2520)
    future_values_10.append(values)
    values_10.append(future_value)
print("Swap value at the end of 10 years: " + str(np.mean(values_10)))
expected_exposure_10, peak_exposure_10, max_peak_exposure_10 = calculate_exposure_metrics(future_values_10)


#For 7 year swap with a notional value of 8 milliion, swap rate equal to initial floating rate
trajectories = 4
values_7 = []
future_values_7 = []
for i in range(trajectories):
    simulated_rates = simulate_vasicek(7, 7*252, history, estimated_alpha)
    future_value, values = find_future_value(simulated_rates, 8000000, 0.02, 0.0003,7*252)
    future_values_7.append(values)
    values_7.append(future_value)
print("Swap value at the end of 7 years: " + str(np.mean(values_7)))
expected_exposure_7, peak_exposure_7, max_peak_exposure_7 = calculate_exposure_metrics(future_values_7)

#For 4 year swap, with a notional value of 6 million, where receiving 3%
trajectories = 4
values_4 = []
future_values_4 = []
for i in range(trajectories):
    simulated_rates = simulate_vasicek(4, 4*252, history, estimated_alpha)
    future_value, values = find_future_value(simulated_rates, 2000000, 0.02, 0.0003,4*252)
    future_values_4.append(values)
    values_4.append(future_value)
print("Swap value at the end of 4 years: " + str(-(np.mean(values_4))))
expected_exposure_4,peak_exposure_4, max_peak_exposure_4 = calculate_exposure_metrics(future_values_4)
print(expected_exposure_4[1000])

Swap value at the end of 10 years: -26914.28440075942
Swap value at the end of 7 years: -130347.49355433756
Swap value at the end of 4 years: 44278.5355987797
268.42992571592004


In [53]:
# Pass cds_spread in terms of fraction not as basis points
def calculate_cva_charge(time_period,R,fixed_r,cds_value,notional,history,alpha,trajectories=1000,float_spread=3e-4):
    
    future_values = []
    for i in range(trajectories):
        simulated_rates = simulate_vasicek(time_period,time_period*252, history, alpha)
        _, values = find_future_value(simulated_rates, notional, fixed_r, float_spread,time_period*252,)
        future_values.append(values)
    exp_exposure,_,_ = calculate_exposure_metrics(future_values)

    cva_value = 0
    
    ##Assuming risk free interest rate as 3.39% referred to US corporate bond rates
    discount_rate = 1/(1+3.39e-2)
    N = len(exp_exposure)
    for i in range(N):
        def_int = cds_value/(1-R)
        def_prob = (np.exp(-def_int*(i-1)/N)-np.exp(-def_int*i/N))
        cva_value += discount_rate*def_prob*exp_exposure[i]
    
    return cva_value*(1-R)

In [272]:
cva_charge = calculate_cva_charge(10,0.32,0.02,423*1e-4,2e6,history,estimated_alpha)
print(cva_charge)

38.9381933209943


## Now using data for different dates and tenor 

## For 10yr interest swap

In [274]:
cva_m10_10yr = calculate_cva_charge(10,0.32,0.02,423*1e-4,2e6,history,estimated_alpha)
cva_m10_5yr = calculate_cva_charge(5,0.32,0.02,416*1e-4,2e6,history,estimated_alpha)
cva_m10_4yr = calculate_cva_charge(4,0.32,0.02,403*1e-4,2e6,history,estimated_alpha)
cva_m10_3yr = calculate_cva_charge(4,0.32,0.02,412*1e-4,2e6,history,estimated_alpha)
cva_m10_2yr = calculate_cva_charge(2,0.32,0.02,404*1e-4,2e6,history,estimated_alpha)
cva_m10_1yr = calculate_cva_charge(1,0.32,0.02,362*1e-4,2e6,history,estimated_alpha)

cva_m14_10yr = calculate_cva_charge(10,0.32,0.02,522*1e-4,2e6,history,estimated_alpha)
cva_m14_5yr = calculate_cva_charge(5,0.32,0.02,555*1e-4,2e6,history,estimated_alpha)
cva_m14_4yr = calculate_cva_charge(4,0.32,0.02,599*1e-4,2e6,history,estimated_alpha)
cva_m14_3yr = calculate_cva_charge(4,0.32,0.02,654*1e-4,2e6,history,estimated_alpha)
cva_m14_2yr = calculate_cva_charge(2,0.32,0.02,713*1e-4,2e6,history,estimated_alpha)
cva_m14_1yr = calculate_cva_charge(1,0.32,0.02,836*1e-4,2e6,history,estimated_alpha)


## For 7yr interest swap

In [273]:
history[0]

0.00889840437294636

In [277]:
init_rate = history[0]
cva_m10_5yr = calculate_cva_charge(5,0.32,init_rate,416*1e-4,8e6,history,estimated_alpha)
cva_m10_4yr = calculate_cva_charge(4,0.32,init_rate,403*1e-4,8e6,history,estimated_alpha)
cva_m10_3yr = calculate_cva_charge(4,0.32,init_rate,412*1e-4,8e6,history,estimated_alpha)
cva_m10_2yr = calculate_cva_charge(2,0.32,init_rate,404*1e-4,8e6,history,estimated_alpha)
cva_m10_1yr = calculate_cva_charge(1,0.32,init_rate,362*1e-4,8e6,history,estimated_alpha)

cva_m14_5yr = calculate_cva_charge(5,0.32,init_rate,555*1e-4,8e6,history,estimated_alpha)
cva_m14_4yr = calculate_cva_charge(4,0.32,init_rate,599*1e-4,8e6,history,estimated_alpha)
cva_m14_3yr = calculate_cva_charge(4,0.32,init_rate,654*1e-4,8e6,history,estimated_alpha)
cva_m14_2yr = calculate_cva_charge(2,0.32,init_rate,713*1e-4,8e6,history,estimated_alpha)
cva_m14_1yr = calculate_cva_charge(1,0.32,init_rate,836*1e-4,8e6,history,estimated_alpha)

## For 4yr interest rate

In [54]:
cva_m10_4yr = calculate_cva_charge(4,0.32,0.03,403*1e-4,-6e6,history,estimated_alpha)
cva_m10_3yr = calculate_cva_charge(3,0.32,0.03,412*1e-4,-6e6,history,estimated_alpha)
cva_m10_2yr = calculate_cva_charge(2,0.32,0.03,404*1e-4,-6e6,history,estimated_alpha)
cva_m10_1yr = calculate_cva_charge(1,0.32,0.03,362*1e-4,-6e6,history,estimated_alpha)

cva_m14_4yr = calculate_cva_charge(4,0.32,0.03,599*1e-4,-6e6,history,estimated_alpha)
cva_m14_3yr = calculate_cva_charge(3,0.32,0.03,654*1e-4,-6e6,history,estimated_alpha)
cva_m14_2yr = calculate_cva_charge(2,0.32,0.03,713*1e-4,-6e6,history,estimated_alpha)
cva_m14_1yr = calculate_cva_charge(1,0.32,0.03,836*1e-4,-6e6,history,estimated_alpha)

In [66]:
#Let us do the simulation for two years and we are interested in finding the exposure and cva for tenor value of 2yrs
trajectories = 1000
future_values_10 = []
T = 2
for i in range(trajectories):
    simulated_rates = simulate_vasicek(T, 252*T, history, estimated_alpha)
    _, values = find_future_value(simulated_rates, 2000000, 0.02, 0.0003,252*T)
    future_values_10.append(values)
#print(len(future_values_10[0]))
future_values_7 = []
T=2
for i in range(trajectories):
    simulated_rates = simulate_vasicek(T, 252*T, history, estimated_alpha)
    _, values = find_future_value(simulated_rates, 8000000, history[0], 0.0003,252*T)
    future_values_7.append(values)
#print(len(future_values_7[0]))

future_values_4 = []
T=2
for i in range(trajectories):
    simulated_rates = simulate_vasicek(T, 252*T, history, estimated_alpha)
    _, values = find_future_value(simulated_rates, -6000000,0.03, 0.0003,252*T)
    future_values_4.append(values)
#print(len(future_values_4[0]))

future_values = [[0 for _ in range(T*252)] for _ in range(trajectories)]
for i in range(trajectories):
    for j in range(T*252):
        #print(i,j)
        future_values[i][j] = future_values_4[i][j]+future_values_10[i][j]+future_values_7[i][j]


## With no collateral and netting

In [32]:
exp_exposure, peak_exposure, max_peak_exposure = calculate_exposure_metrics(future_values)
discount_rate = 1/(1+3.39e-2)
N = len(exp_exposure)
cva_value = 0
cds_value = 443e-4
R = 0.32
for i in range(N):
    def_int = cds_value/(1-R)
    def_prob = (np.exp(-def_int*(i-1)/N)-np.exp(-def_int*i/N))
    cva_value += discount_rate*def_prob*exp_exposure[i]

## Without Netting and collateral

In [50]:
no_traj = len(future_values_10)
traj_len = len(future_values_10[0])
C = 1e5
peak_exposure = np.zeros(traj_len)
exp_exposure = np.zeros(traj_len)
for i in range(traj_len):
    temp_store = []
    for j in range(no_traj):
        t = max(future_values_10[j][i]-C,0)+max(future_values_4[j][i]-C,0)+max(future_values_7[j][i]-C,0)
        temp_store.append(t)
    exp_exposure[i] = 1.0*sum(temp_store)/no_traj
    peak_exposure[i] = float(np.percentile(np.array(temp_store),97.5))
discount_rate = 1/(1+3.39e-2)
N = len(exp_exposure)
cva_value = 0
cds_value = 443e-4
R = 0.32
for i in range(N):
    def_int = cds_value/(1-R)
    def_prob = (np.exp(-def_int*(i-1)/N)-np.exp(-def_int*i/N))
    cva_value += discount_rate*def_prob*exp_exposure[i]

## With Netting and total Collateral

In [61]:
exp_exposure, peak_exposure, max_peak_exposure = calculate_exposure_metrics([[value-1e5 for value in arr] for arr in future_values])
discount_rate = 1/(1+3.39e-2)
N = len(exp_exposure)
cva_value = 0
cds_value = 443e-4
R = 0.32
for i in range(N):
    def_int = cds_value/(1-R)
    def_prob = (np.exp(-def_int*(i-1)/N)-np.exp(-def_int*i/N))
    cva_value += discount_rate*def_prob*exp_exposure[i]